In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 300
pd.set_option('display.max_columns', None)
import math
import datetime as dt
import itertools
import random
import scipy.stats as stats
import seaborn as sns

# # Subtype + Time Regulation

In [ ]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/02.AUMCdb_SAKI_trajCluster/df_mixAK_fea3_C3_aumc.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("Number of included patients:",len(all_lst))

df_type_filt["groupHPD"].value_counts()

In [3]:
data_dir = "/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_aumc/disease_definition/AKI/"
infile = data_dir + "aumcdb_sk_event_time.csv"
df_time = pd.read_csv(infile)
df_time = df_time[df_time["stay_id"].isin(all_lst)]
df_time = df_time[["stay_id","saki_onset"]]
#df_time["saki_onset"] = pd.to_datetime(df_time["saki_onset"])
df_time.loc[:, 'saki_onset_a1d'] = df_time.saki_onset + 24 #+ dt.timedelta(hours=24)
df_time = pd.merge(df_type_filt,df_time,how="inner",on="stay_id")
df_time.head(2)

,stay_id,groupHPD,saki_onset,saki_onset_a1d
0,5,2,3.0,27.0
1,15,2,3.0,27.0


# Vasopressors

In [4]:
df_vaso = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb//clean/medication/vaso/vaso_clean.csv")
df_vaso["starttime"] = df_vaso["start_time"]/60 #hour
df_vaso["endtime"] = df_vaso["stop_time"]/60 #hour
df_vaso = df_vaso[['admissionid','starttime',  'gamma', 'drug']]
df_vaso.columns = ['stay_id','starttime',  'amount', 'drug']
df_vaso = pd.merge(df_vaso,df_time,how="right",on="stay_id")

df_vaso= df_vaso[['stay_id', 'starttime', 'drug', 'amount',  'groupHPD']]
#df_vaso = df_vaso[(df_vaso["charttime"]<=df_vaso["saki_onset_a1d"])&(df_vaso["charttime"]>=df_vaso["saki_onset"])]
df_vaso.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,5,0.783333,norepinephrine,0.166667,2


In [ ]:
# Drug Utilization Rate
# df_sum = df_vaso.groupby(["stay_id","drug"]).agg({"dose":"sum"}).reset_index()
# df_sum["is_vaso"] = 1
# df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_vaso").reset_index()
# df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
# df_stat_01 = df_stat_01.fillna(0)
# df_stat_01.groupby('groupHPD').mean()*100

In [ ]:
# Average drug usage
# df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="dose").reset_index()
# df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
# df_stat_amount.groupby('groupHPD').mean().reset_index()

# Antibiotics

In [7]:
df_anti = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb//clean/medication/anti/antibiotics.csv")
df_anti["starttime"] = df_anti["start"]/1000/60/60 #hour
df_anti["endtime"] = df_anti["stop"]/1000/60/60 #hour
df_anti = df_anti[['admissionid',  'item',  'administered', 'administeredunit','starttime', 'endtime']]
df_anti.columns = ["stay_id","drug","dose","doseunit",'starttime', 'endtime']
df_anti.loc[df_anti['doseunit'] == 'g', 'dose'] *= 1000
df_anti.loc[df_anti['doseunit'] == 'g', 'doseunit'] = 'mg'

df_anti = pd.merge(df_anti,df_time,how="right",on="stay_id")
#df_anti = df_anti[(df_anti["starttime"]<=df_anti["saki_onset_a1d"])&(df_anti["starttime"]>=df_anti["saki_onset"])]
df_anti = df_anti[["stay_id","starttime", "drug","dose","groupHPD"]]
df_anti['drug'] = df_anti['drug'].str.replace(r'\(.*\)', '')
df_anti.head(1)

<ipython-input-7-ffbc034e904b>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_anti['drug'] = df_anti['drug'].str.replace(r'\(.*\)', '')


,stay_id,starttime,drug,dose,groupHPD
0,5,1.066667,Ciprofloxacine,400.0,2


In [ ]:
# Drug Utilization Rate
df_sum = df_anti.groupby(["stay_id","drug"]).agg({"groupHPD":"sum"}).reset_index()
df_sum["is_anti"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_anti").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01[df_stat_01 > 5].dropna(axis=1, how='all')


,stay_id,Aciclovir,Amoxicilline,Amoxicilline/Clavulaanzuur,Cefazoline,Cefotaxim,Ceftazidim,Ceftriaxon,Ciprofloxacine,Co-Trimoxazol,Ecalta,Erythromycine,Flucloxacilline,Fluconazol,Gentamicine,Imipenem,Levofloxacine,Meropenem,Metronidazol,Mupirocine zalf,Piperacilline,Tobramycine,Vancomycine,Velosuline,Voriconazol,X nader te bepalen
groupHPD,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.209961e+06,9.219858,12.765957,14.007092,15.780142,31.560284,5.141844,65.425532,23.049645,11.524823,NaN,33.156028,10.283688,15.602837,9.751773,9.751773,7.092199,NaN,25.886525,NaN,6.028369,7.801418,34.042553,27.304965,5.673759,21.985816
2,1.176900e+06,NaN,NaN,14.000000,19.714286,36.785714,NaN,60.000000,14.214286,8.000000,NaN,31.571429,11.071429,6.500000,6.857143,NaN,6.571429,NaN,16.785714,5.857143,NaN,NaN,19.142857,25.571429,NaN,17.500000
3,1.124017e+06,12.328767,9.589041,9.132420,13.698630,33.789954,6.392694,56.621005,21.917808,11.415525,7.305936,40.639269,8.675799,10.958904,6.392694,9.132420,10.045662,8.675799,25.570776,5.022831,10.502283,12.328767,30.136986,21.004566,9.132420,15.981735


# fluid in

In [9]:
df_fluid = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb//clean/treatment/AUMCdb_fluid_NoNut.csv")
df_fluid = df_fluid[df_fluid["amount"]<5000]
df_fluid = df_fluid[['stay_id','starttime','endtime','item','amount']]
df_fluid.columns = ["stay_id",'starttime','endtime',"drug",'amount']
df_fluid["starttime"] = df_fluid["starttime"]/60
df_fluid["endtime"] = df_fluid["endtime"]/60

df_fluid = pd.merge(df_fluid,df_time,how="right",on="stay_id")
#df_fluid = df_fluid[(df_fluid["charttime"]<=df_fluid["saki_onset_a1d"])&(df_fluid["charttime"]>=df_fluid["saki_onset"])]
df_fluid =  df_fluid[["stay_id","starttime","drug","amount","groupHPD"]]
df_fluid.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,5,0.7,Ri-Lac (Ringers lactaat),500.0,2


In [ ]:
# Drug utilization rate
df_sum = df_fluid.groupby(["stay_id"]).agg({"amount":"sum"}).reset_index()
df_sum["drug"] ="fluid"
df_sum["is_fluid"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_fluid").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01.groupby('groupHPD').mean()*100

,stay_id,fluid
groupHPD,,
1,1.209961e+06,100.0
2,1.176900e+06,100.0
3,1.124017e+06,100.0


In [ ]:
# Average drug usage
df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="amount").reset_index()
df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
df_stat_amount.groupby('groupHPD').mean().reset_index()

,groupHPD,stay_id,fluid
0,1,12099.609929,20513.565088
1,2,11769.000714,14013.983193
2,3,11240.168950,18810.845069


# Diuretics

In [12]:
df_di = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb/clean/medication/diuretic/AUMCdb_diuretics.csv")
df_di = df_di[['stay_id',  'drug', 'starttime', 'endtime','administered']]
df_di["starttime"] = df_di["starttime"]/60
df_di["endtime"] = df_di["endtime"]/60
df_di =df_di.rename(columns={ 'administered':'amount'})

df_di = pd.merge(df_di,df_time,how="right",on="stay_id")
# df_di = df_di[(df_di["starttime"]<=df_di["saki_onset_a1d"])&(df_di["starttime"]>=df_di["saki_onset"])]
df_di = df_di[[ 'stay_id', 'starttime',"drug", 'amount',"groupHPD" ]]
map_dict = {'Furosemide (Lasix)':'Furosemide', 'Bumetanide (Burinex)':'Bumetanide'}
df_di["drug"] = df_di["drug"].map(map_dict)
df_di.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,5,42.833333,Furosemide,20.0,2


In [13]:
df_sum = df_di.groupby(["stay_id","drug"]).agg({"amount":"mean"}).reset_index()
df_sum.head(1)

,stay_id,drug,amount
0,5,Furosemide,20.0


In [ ]:
# Drug Utilization Rate
df_sum["is_diuretic"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_diuretic").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01.groupby('groupHPD').mean()*100

,stay_id,Bumetanide,Furosemide
groupHPD,,,
1,1.209961e+06,0.531915,78.900709
2,1.176900e+06,0.357143,68.214286
3,1.124017e+06,0.913242,65.296804


In [ ]:
# Average drug usage
df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="amount").reset_index()
df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
df_stat_amount.groupby('groupHPD').mean().reset_index()

,groupHPD,stay_id,Bumetanide,Furosemide
0,1,11904.755056,4.460833,60.502662
1,2,11793.091099,2.641778,33.952374
2,3,11820.398601,1.704167,76.436786


# Other Drugs

In [16]:
df_ot = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb/raw/AmsterdamUMCdb-v1/drugitems.csv", encoding='latin-1')
df_ot["starttime"] = df_ot["start"]/1000/60/60 #hour
df_ot["endtime"] = df_ot["stop"]/1000/60/60 #hour
df_ot =df_ot[['admissionid','starttime', 'endtime','ordercategory','item', 'administered', 'administeredunit']]
df_ot.columns = ['stay_id','starttime', 'endtime','ordercategory','drug','amount','doseunit']
df_ot['drug'] = df_ot['drug'].str.replace(r'\(.*\)', '')
df_ot['drug'] = df_ot['drug'].str.rstrip(' 0123456789/-.%')
df_ot = pd.merge(df_ot,df_time,how="right",on="stay_id")
# df_ot = df_ot[(df_ot["starttime"]<=df_ot["saki_onset_a1d"])&(df_ot["starttime"]>=df_ot["saki_onset"])]
df_ot = df_ot[[ 'stay_id', 'starttime', "drug", 'amount',"groupHPD" ]]
df_ot.head(1)

<ipython-input-16-1aea719228a0>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ot['drug'] = df_ot['drug'].str.replace(r'\(.*\)', '')


,stay_id,starttime,drug,amount,groupHPD
0,5,1.066667,Ciprofloxacine,400.0,2


In [ ]:
df_sum = df_ot.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()

# Drug Utilization Rate - Overall
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
# Calculate the overall medication rate
count_ones = df_stat_01.eq(1).sum()
df_stat_01_all = pd.DataFrame(count_ones / len(df_stat_01) * 100).T
df_stat_01_all

,stay_id,1 Roomijs 65ml,1 Waterijs,1 bakje eiersalade,1 bakje roomkaas met bieslook,1 bakje zalmsalade,1 cupje margarine /roomboter,1 gekookt ei,1 knäckebröd/beschuit/toast,1 kroket/ pizzapunt,1 loempia/ kip/ham,"1 mini twix,mars,snicker,mailky way",1 omelet/roerei,1 pannekoek,1 pindakaas,1 plak cake,1 plak kaas,1 plak vleeswaar,1 portie aardappelen/aardappelpuree,1 portie appelmoes/vruchtenmoes,1 portie cornflakes,1 portie fruit,1 portie gebakken aardappelen,1 portie gehakt of kip in pindasaus,1 portie gewelde pruimen,1 portie groente,1 portie jus,1 portie macaroni/spaghetti,1 portie makreel,1 portie mixmaaltijd,1 portie muesli,1 portie nagerecht,1 portie ragout met kalfsvlees,1 portie rauwkost,1 portie spritsbrokken,1 portie stamppot zonder vlees,1 portie vis,1 portie vlees,1 portie witte rijst/zilvervliesrijst,1 portie worst,1 smeerkaas,1 snee brood zonder korst,1 snee ontbijtkoek,1 snee wit-bruin-volkoren-krente-roggebrood,1 zakje M&M,1 zakje knabbelnoten,1 zakje poffertjes,1 zoet beleg,1/2 Waterijs,1/2 Roomijs,1/2 bakje eiersalade,1/2 bakje zalmsalade,1/2 cupje margarine /roomboter,1/2 gekookt ei,1/2 haring,1/2 knäckebröd/beschuit/toast,1/2 kroket/ pizzapunt,"1/2 mini twix,mars,snicker,mailky way",1/2 omelet/roerei,1/2 pannekoek,1/2 pindakaas,1/2 plak cake,1/2 plak kaas,1/2 plak vleeswaar,1/2 portie aardappelen/aardappelpuree,1/2 portie appelmoes/vruchtenmoes,1/2 portie bami,1/2 portie cornflakes,1/2 portie fruit,1/2 portie gebakken aardappelen,1/2 portie gehakt of kip in pindasaus,1/2 portie gewelde pruimen,1/2 portie groente,1/2 portie jus,1/2 portie macaroni/spaghetti,1/2 portie makreel,1/2 portie mixmaaltijd,1/2 portie muesli,1/2 portie nagerecht,1/2 portie nasi,1/2 portie rauwkost,1/2 portie stamppot zonder vlees,1/2 portie vis,1/2 portie vlees,1/2 portie witte rijst/zilvervliesrijst,1/2 smeerkaas,1/2 snee brood zonder korst,1/2 snee ontbijtkoek,1/2 snee wit-bruin-volkoren-krente-roggebrood,1/2 zoet beleg,1/4 Waterijs,1/4 Roomijs,1/4 bakje eiersalade,1/4 bakje zalmsalade,1/4 cupje margarine /roomboter,1/4 gekookt ei,1/4 knäckebröd/beschuit/toast,1/4 kroket/ pizzapunt,1/4 loempia/ kip/ham,"1/4 mini twix,mars,snicker,mailky way",1/4 omelet/roerei,1/4 pakje sultana/ balisto,1/4 pannekoek,1/4 pindakaas,1/4 plak cake,1/4 plak kaas,1/4 plak vleeswaar,1/4 portie aardappelen/aardappelpuree,1/4 portie appelmoes/vruchtenmoes,1/4 portie bami,1/4 portie cornflakes,1/4 portie fruit,1/4 portie gebakken aardappelen,1/4 portie gewelde pruimen,1/4 portie groente,1/4 portie jus,1/4 portie macaroni/spaghetti,1/4 portie mixmaaltijd,1/4 portie muesli,1/4 portie nagerecht,1/4 portie nasi,1/4 portie ragout met kalfsvlees,1/4 portie rauwkost,1/4 portie stamppot zonder vlees,1/4 portie vis,1/4 portie vlees,1/4 portie witte rijst/zilvervliesrijst,1/4 portie worst,1/4 smeerkaas,1/4 snee brood zonder korst,1/4 snee ontbijtkoek,1/4 snee wit-bruin-volkoren-krente-roggebrood,1/4 zoet beleg,2 Vissticks,3/4 Waterijs,3/4 Roomijs,3/4 bakje eiersalade,3/4 cupje margarine /roomboter,3/4 gekookt ei,3/4 knäckebröd/beschuit/toast,3/4 kroket/ pizzapunt,3/4 omelet/roerei,3/4 plak cake,3/4 plak kaas,3/4 plak vleeswaar,3/4 portie aardappelen/aardappelpuree,3/4 portie appelmoes/vruchtenmoes,3/4 portie bami,3/4 portie cornflakes,3/4 portie fruit,3/4 portie gebakken aardappelen,3/4 portie groente,3/4 portie jus,3/4 portie macaroni/spaghetti,3/4 portie mixmaaltijd,3/4 portie muesli,3/4 portie nagerecht,3/4 portie rauwkost,3/4 portie spritsbrokken,3/4 portie stamppot zonder vlees,3/4 portie vis,3/4 portie vlees,3/4 portie witte rijst/zilvervliesrijst,3/4 smeerkaas,3/4 snee brood zonder korst,3/4 snee wit-bruin-volkoren-krente-roggebrood,3/4 zoet beleg,ABLE - Studie,ACCESS - Studie,ART-123 studie,Abciximab,Acenocoumarol,Acetazolamide,Acetrizoaat meglumine,Acetylcysteine,Acetylsalicylzuur,Aciclovir,Aciclovir oculair,Actrapid,Adenosine,Adrenaline,Alarm Bleeding - Studie,Albumine,Alendroninezuur,Alfacalcidol,Alfuzosine,Allopurinol,Alprazolam,Alteplase,Amfotericine B,Am

In [ ]:
# Drug Utilization Rate - Subtype
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01.head()

,stay_id,1 Roomijs 65ml,1 Waterijs,1 bakje eiersalade,1 bakje roomkaas met bieslook,1 bakje zalmsalade,1 cupje margarine /roomboter,1 gekookt ei,1 knäckebröd/beschuit/toast,1 kroket/ pizzapunt,1 loempia/ kip/ham,"1 mini twix,mars,snicker,mailky way",1 omelet/roerei,1 pannekoek,1 pindakaas,1 plak cake,1 plak kaas,1 plak vleeswaar,1 portie aardappelen/aardappelpuree,1 portie appelmoes/vruchtenmoes,1 portie cornflakes,1 portie fruit,1 portie gebakken aardappelen,1 portie gehakt of kip in pindasaus,1 portie gewelde pruimen,1 portie groente,1 portie jus,1 portie macaroni/spaghetti,1 portie makreel,1 portie mixmaaltijd,1 portie muesli,1 portie nagerecht,1 portie ragout met kalfsvlees,1 portie rauwkost,1 portie spritsbrokken,1 portie stamppot zonder vlees,1 portie vis,1 portie vlees,1 portie witte rijst/zilvervliesrijst,1 portie worst,1 smeerkaas,1 snee brood zonder korst,1 snee ontbijtkoek,1 snee wit-bruin-volkoren-krente-roggebrood,1 zakje M&M,1 zakje knabbelnoten,1 zakje poffertjes,1 zoet beleg,1/2 Waterijs,1/2 Roomijs,1/2 bakje eiersalade,1/2 bakje zalmsalade,1/2 cupje margarine /roomboter,1/2 gekookt ei,1/2 haring,1/2 knäckebröd/beschuit/toast,1/2 kroket/ pizzapunt,"1/2 mini twix,mars,snicker,mailky way",1/2 omelet/roerei,1/2 pannekoek,1/2 pindakaas,1/2 plak cake,1/2 plak kaas,1/2 plak vleeswaar,1/2 portie aardappelen/aardappelpuree,1/2 portie appelmoes/vruchtenmoes,1/2 portie bami,1/2 portie cornflakes,1/2 portie fruit,1/2 portie gebakken aardappelen,1/2 portie gehakt of kip in pindasaus,1/2 portie gewelde pruimen,1/2 portie groente,1/2 portie jus,1/2 portie macaroni/spaghetti,1/2 portie makreel,1/2 portie mixmaaltijd,1/2 portie muesli,1/2 portie nagerecht,1/2 portie nasi,1/2 portie rauwkost,1/2 portie stamppot zonder vlees,1/2 portie vis,1/2 portie vlees,1/2 portie witte rijst/zilvervliesrijst,1/2 smeerkaas,1/2 snee brood zonder korst,1/2 snee ontbijtkoek,1/2 snee wit-bruin-volkoren-krente-roggebrood,1/2 zoet beleg,1/4 Waterijs,1/4 Roomijs,1/4 bakje eiersalade,1/4 bakje zalmsalade,1/4 cupje margarine /roomboter,1/4 gekookt ei,1/4 knäckebröd/beschuit/toast,1/4 kroket/ pizzapunt,1/4 loempia/ kip/ham,"1/4 mini twix,mars,snicker,mailky way",1/4 omelet/roerei,1/4 pakje sultana/ balisto,1/4 pannekoek,1/4 pindakaas,1/4 plak cake,1/4 plak kaas,1/4 plak vleeswaar,1/4 portie aardappelen/aardappelpuree,1/4 portie appelmoes/vruchtenmoes,1/4 portie bami,1/4 portie cornflakes,1/4 portie fruit,1/4 portie gebakken aardappelen,1/4 portie gewelde pruimen,1/4 portie groente,1/4 portie jus,1/4 portie macaroni/spaghetti,1/4 portie mixmaaltijd,1/4 portie muesli,1/4 portie nagerecht,1/4 portie nasi,1/4 portie ragout met kalfsvlees,1/4 portie rauwkost,1/4 portie stamppot zonder vlees,1/4 portie vis,1/4 portie vlees,1/4 portie witte rijst/zilvervliesrijst,1/4 portie worst,1/4 smeerkaas,1/4 snee brood zonder korst,1/4 snee ontbijtkoek,1/4 snee wit-bruin-volkoren-krente-roggebrood,1/4 zoet beleg,2 Vissticks,3/4 Waterijs,3/4 Roomijs,3/4 bakje eiersalade,3/4 cupje margarine /roomboter,3/4 gekookt ei,3/4 knäckebröd/beschuit/toast,3/4 kroket/ pizzapunt,3/4 omelet/roerei,3/4 plak cake,3/4 plak kaas,3/4 plak vleeswaar,3/4 portie aardappelen/aardappelpuree,3/4 portie appelmoes/vruchtenmoes,3/4 portie bami,3/4 portie cornflakes,3/4 portie fruit,3/4 portie gebakken aardappelen,3/4 portie groente,3/4 portie jus,3/4 portie macaroni/spaghetti,3/4 portie mixmaaltijd,3/4 portie muesli,3/4 portie nagerecht,3/4 portie rauwkost,3/4 portie spritsbrokken,3/4 portie stamppot zonder vlees,3/4 portie vis,3/4 portie vlees,3/4 portie witte rijst/zilvervliesrijst,3/4 smeerkaas,3/4 snee brood zonder korst,3/4 snee wit-bruin-volkoren-krente-roggebrood,3/4 zoet beleg,ABLE - Studie,ACCESS - Studie,ART-123 studie,Abciximab,Acenocoumarol,Acetazolamide,Acetrizoaat meglumine,Acetylcysteine,Acetylsalicylzuur,Aciclovir,Aciclovir oculair,Actrapid,Adenosine,Adrenaline,Alarm Bleeding - Studie,Albumine,Alendroninezuur,Alfacalcidol,Alfuzosine,Allopurinol,Alprazolam,Alteplase,Amfotericine B,Am

In [ ]:
# Merge
df_stat_01_add = pd.concat([df_stat_01_all,df_stat_01],axis=0)
df_stat_01_add["groupHPD"] = [0,1,2,3]
# Remove drugs that are less than 10% of the total SA-AKI patients
first_row = df_stat_01_add.iloc[0]
columns_to_drop = first_row[first_row <= 10].index
df_stat_01_add = df_stat_01_add.drop(columns=columns_to_drop)
df_stat_01_add["groupHPD"] = [0,1,2,3]
df_stat_01_add = df_stat_01_add[df_stat_01_add.columns[df_stat_01_add.iloc[0].argsort()[::-1]]]
df_stat_01_add 

,Drukzak,"NaCL 0,9% spuit","NaCl 0,45%/Glucose 2,",Magnesiumsulfaat,Propofol,Fentanyl,"NaCl 0,",Nadroparine,Noradrenaline,Kaliumchloride,Paracetamol,Furosemide,SDD drank,SDD pasta,Gelofusine,Midazolam,Ceftriaxon,Nutrison Protein Plus,Thiamine,Cernevit Multivitaminen,Gefiltreerde Ery's,Macrogol X,Haloperidol,Pantoprazol,Kaliumchloride drank,Hydrocortison,Glucose,Spoorelementen,Hypromellose,Macrogol,Water 150ml,Natrium-kalium-fosfaat,Heparine,Oxazepam,Cefotaxim,Xylometazoline neusspray,Medicijnlijn medicatie,Ri-Lac,Actrapid,Lorazepam,Metoprolol,Erythromycine,Morfine,Fosfaatklysma,Carbasalaatcalcium,Ventolin:Atrovent 1:,Calcium Glubionaat,Amiodaron Onderhoudsdosis,Rocuronium,Dopamine,Enoximon,Ventolin:Atrovent:Mucomyst 1:1:,Velosuline,Nutrison Standaard,Amiodaron Oplaaddosis,Insuline aspart,Nitroglycerine,Clonidine,Vancomycine,Nutrison Sterilized water,Water 100ml,Temazepam,Fytomenadion,Captopril,Metronidazol,Supradyn,X nader te bepalen,Cefazoline,Ascorbinezuur,Fresh Frozen Plasma,Glycerofosfaat,Water 50ml,Ciprofloxacine,Acetazolamide,Metoclopram,Amlodipine,Vitamine B Complex,Oculentum simplex oogzalf,Dobutamine,Etomidaat,Glucose 5% spuit,Vla 150ml,Thrombocyten suspensie,Fluticason neusspray,Albumine,Thee 150ml,Tranexaminezuur,Water 200ml,Amoxicilline/Clavulaanzuur,Simvastatine,Lisinopril,Magnesiumoxide,"Natrium bicarbonaat 8,",Nicardipine,Digoxine,Flucloxacilline,Clopidogrel,groupHPD
0,96.701787,92.670637,88.410444,88.273019,88.089785,84.241869,80.897847,79.111315,78.744847,75.858910,73.889143,70.682547,68.300504,68.254695,65.048099,61.245992,61.062758,57.077416,55.153459,48.373798,47.136967,45.579478,43.334860,43.243243,43.197435,42.098030,40.677966,40.586349,40.311498,38.295923,36.509391,36.371965,36.326157,36.280348,35.135135,34.906093,34.631241,33.531837,33.486028,33.119560,33.119560,32.890518,30.691709,28.905176,28.813559,27.668346,27.622538,27.118644,26.385708,26.294091,26.248282,25.973431,25.561154,25.423729,24.874027,24.644984,24.370133,24.186899,24.095282,23.957856,22.995877,21.713239,21.209345,20.384792,20.018323,19.239579,18.918919,18.094366,18.094366,18.048557,18.002749,17.453046,17.269812,17.224004,16.949153,16.857536,16.032982,15.712322,15.666514,15.300046,15.162620,15.116812,15.025195,14.567109,14.109024,13.971599,13.971599,13.834173,13.513514,13.192854,13.009620,12.001832,11.818598,11.543747,10.627577,10.627577,10.169492,0
1,95.921986,94.148936,89.007092,90.602837,85.638298,88.652482,85.106383,79.432624,86.170213,80.673759,72.340426,78.900709,70.744681,71.099291,70.921986,70.035461,65.425532,56.028369,62.234043,53.900709,59.042553,51.595745,49.468085,55.496454,46.808511,54.787234,55.319149,44.503546,46.276596,35.815603,41.134752,40.957447,45.921986,37.588652,31.560284,40.780142,39.716312,30.319149,37.056738,38.120567,35.638298,33.156028,28.546099,29.787234,30.141844,28.368794,34.042553,36.879433,29.078014,29.255319,37.056738,24.822695,27.304965,29.787234,33.510638,23.049645,26.418440,23.049645,34.042553,32.801418,23.404255,25.000000,31.205674,20.035461,25.886525,23.226950,22.340426,15.780142,18.085106,24.645390,14.893617,18.262411,23.049645,20.035461,16.843972,21.276596,14.716312,21.631206,19.326241,16.489362,19.858156,17.553191,22.695035,20.390071,25.000000,15.957447,17.375887,16.843972,14.007092,13.829787,13.652482,14.539007,21.099291,14.716312,13.475177,10.283688,10.106383,1
2,96.928571,91.857143,88.214286,87.214286,88.571429,81.214286,78.214286,81.428571,73.571429,74.500000,77.285714,68.214286,65.500000,65.428571,62.500000,55.357143,60.000000,57.857143,50.428571,43.928571,40.500000,43.642857,43.857143,37.142857,44.714286,32.857143,31.857143,36.857143,37.142857,38.928571,36.285714,36.000000,31.428571,38.000000,36.785714,32.785714,33.000000,34.214286,31.500000,32.928571,33.500000,31.571429,33.071429,30.357143,28.357143,28.357143,23.357143,20.214286,23.357143,24.714286,19.642857,27.285714,25.571429,24.785714,18.785714,24.357143,23.785714,26.000000,19.142857,20.857143,23.714286,21.500000,14.714286,21.642857,16.78

# Merge
- High-frequency drugs
- High-dose drugs

In [112]:
df_merge = pd.concat([df_anti,df_fluid,df_di,df_ot],axis=0) #df_vaso,
# def custom_map(value):
#     return aumc_to_mimic_mapping.get(value, value)
# df_merge["drug"] = df_merge["drug"].map(custom_map)
df_merge['drug'] = df_merge['drug'].str.replace(r'\(.*\)', '')
df_merge['drug'] = df_merge['drug'].str.rstrip(' 0123456789/-.%')
df_merge['drug'] = df_merge['drug'].str.rstrip(',')

df_sum = df_merge.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()
df_sum.drug = df_sum.drug.str.lower()
df_sum.head(1)

<ipython-input-112-655ed0065306>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merge['drug'] = df_merge['drug'].str.replace(r'\(.*\)', '')


,stay_id,drug,amount
0,5,1/4 portie aardappelen/aardappelpuree,0.25


In [ ]:
# Drug Utilization Rate - Overall
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)

# Calculate the overall medication rate
count_ones = df_stat_01.eq(1).sum()
df_stat_01_all = pd.DataFrame(count_ones / len(df_stat_01) * 100).T
df_stat_01_all

,stay_id,1 bakje eiersalade,1 bakje roomkaas met bieslook,1 bakje zalmsalade,1 cupje margarine /roomboter,1 gekookt ei,1 knäckebröd/beschuit/toast,1 kroket/ pizzapunt,1 loempia/ kip/ham,"1 mini twix,mars,snicker,mailky way",1 omelet/roerei,1 pannekoek,1 pindakaas,1 plak cake,1 plak kaas,1 plak vleeswaar,1 portie aardappelen/aardappelpuree,1 portie appelmoes/vruchtenmoes,1 portie cornflakes,1 portie fruit,1 portie gebakken aardappelen,1 portie gehakt of kip in pindasaus,1 portie gewelde pruimen,1 portie groente,1 portie jus,1 portie macaroni/spaghetti,1 portie makreel,1 portie mixmaaltijd,1 portie muesli,1 portie nagerecht,1 portie ragout met kalfsvlees,1 portie rauwkost,1 portie spritsbrokken,1 portie stamppot zonder vlees,1 portie vis,1 portie vlees,1 portie witte rijst/zilvervliesrijst,1 portie worst,1 roomijs 65ml,1 smeerkaas,1 snee brood zonder korst,1 snee ontbijtkoek,1 snee wit-bruin-volkoren-krente-roggebrood,1 waterijs,1 zakje knabbelnoten,1 zakje m&m,1 zakje poffertjes,1 zoet beleg,1/2 waterijs,1/2 bakje eiersalade,1/2 bakje zalmsalade,1/2 cupje margarine /roomboter,1/2 gekookt ei,1/2 haring,1/2 knäckebröd/beschuit/toast,1/2 kroket/ pizzapunt,"1/2 mini twix,mars,snicker,mailky way",1/2 omelet/roerei,1/2 pannekoek,1/2 pindakaas,1/2 plak cake,1/2 plak kaas,1/2 plak vleeswaar,1/2 portie aardappelen/aardappelpuree,1/2 portie appelmoes/vruchtenmoes,1/2 portie bami,1/2 portie cornflakes,1/2 portie fruit,1/2 portie gebakken aardappelen,1/2 portie gehakt of kip in pindasaus,1/2 portie gewelde pruimen,1/2 portie groente,1/2 portie jus,1/2 portie macaroni/spaghetti,1/2 portie makreel,1/2 portie mixmaaltijd,1/2 portie muesli,1/2 portie nagerecht,1/2 portie nasi,1/2 portie rauwkost,1/2 portie stamppot zonder vlees,1/2 portie vis,1/2 portie vlees,1/2 portie witte rijst/zilvervliesrijst,1/2 roomijs,1/2 smeerkaas,1/2 snee brood zonder korst,1/2 snee ontbijtkoek,1/2 snee wit-bruin-volkoren-krente-roggebrood,1/2 zoet beleg,1/4 waterijs,1/4 bakje eiersalade,1/4 bakje zalmsalade,1/4 cupje margarine /roomboter,1/4 gekookt ei,1/4 knäckebröd/beschuit/toast,1/4 kroket/ pizzapunt,1/4 loempia/ kip/ham,"1/4 mini twix,mars,snicker,mailky way",1/4 omelet/roerei,1/4 pakje sultana/ balisto,1/4 pannekoek,1/4 pindakaas,1/4 plak cake,1/4 plak kaas,1/4 plak vleeswaar,1/4 portie aardappelen/aardappelpuree,1/4 portie appelmoes/vruchtenmoes,1/4 portie bami,1/4 portie cornflakes,1/4 portie fruit,1/4 portie gebakken aardappelen,1/4 portie gewelde pruimen,1/4 portie groente,1/4 portie jus,1/4 portie macaroni/spaghetti,1/4 portie mixmaaltijd,1/4 portie muesli,1/4 portie nagerecht,1/4 portie nasi,1/4 portie ragout met kalfsvlees,1/4 portie rauwkost,1/4 portie stamppot zonder vlees,1/4 portie vis,1/4 portie vlees,1/4 portie witte rijst/zilvervliesrijst,1/4 portie worst,1/4 roomijs,1/4 smeerkaas,1/4 snee brood zonder korst,1/4 snee ontbijtkoek,1/4 snee wit-bruin-volkoren-krente-roggebrood,1/4 zoet beleg,2 vissticks,3/4 waterijs,3/4 bakje eiersalade,3/4 cupje margarine /roomboter,3/4 gekookt ei,3/4 knäckebröd/beschuit/toast,3/4 kroket/ pizzapunt,3/4 omelet/roerei,3/4 plak cake,3/4 plak kaas,3/4 plak vleeswaar,3/4 portie aardappelen/aardappelpuree,3/4 portie appelmoes/vruchtenmoes,3/4 portie bami,3/4 portie cornflakes,3/4 portie fruit,3/4 portie gebakken aardappelen,3/4 portie groente,3/4 portie jus,3/4 portie macaroni/spaghetti,3/4 portie mixmaaltijd,3/4 portie muesli,3/4 portie nagerecht,3/4 portie rauwkost,3/4 portie spritsbrokken,3/4 portie stamppot zonder vlees,3/4 portie vis,3/4 portie vlees,3/4 portie witte rijst/zilvervliesrijst,3/4 roomijs,3/4 smeerkaas,3/4 snee brood zonder korst,3/4 snee wit-bruin-volkoren-krente-roggebrood,3/4 zoet beleg,abciximab,able - studie,access - studie,acenocoumarol,acetazolamide,acetrizoaat meglumine,acetylcysteine,acetylsalicylzuur,aciclovir,aciclovir oculair,actrapid,adenosine,adrenaline,alarm bleeding - studie,albumine,alendroninezuur,alfacalcidol,alfuzosine,allopurinol,alprazolam,alteplase,amfotericine b,amfotericine b in

In [ ]:
# Drug Utilization Rate - Subtype
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01.head()

,stay_id,1 bakje eiersalade,1 bakje roomkaas met bieslook,1 bakje zalmsalade,1 cupje margarine /roomboter,1 gekookt ei,1 knäckebröd/beschuit/toast,1 kroket/ pizzapunt,1 loempia/ kip/ham,"1 mini twix,mars,snicker,mailky way",1 omelet/roerei,1 pannekoek,1 pindakaas,1 plak cake,1 plak kaas,1 plak vleeswaar,1 portie aardappelen/aardappelpuree,1 portie appelmoes/vruchtenmoes,1 portie cornflakes,1 portie fruit,1 portie gebakken aardappelen,1 portie gehakt of kip in pindasaus,1 portie gewelde pruimen,1 portie groente,1 portie jus,1 portie macaroni/spaghetti,1 portie makreel,1 portie mixmaaltijd,1 portie muesli,1 portie nagerecht,1 portie ragout met kalfsvlees,1 portie rauwkost,1 portie spritsbrokken,1 portie stamppot zonder vlees,1 portie vis,1 portie vlees,1 portie witte rijst/zilvervliesrijst,1 portie worst,1 roomijs 65ml,1 smeerkaas,1 snee brood zonder korst,1 snee ontbijtkoek,1 snee wit-bruin-volkoren-krente-roggebrood,1 waterijs,1 zakje knabbelnoten,1 zakje m&m,1 zakje poffertjes,1 zoet beleg,1/2 waterijs,1/2 bakje eiersalade,1/2 bakje zalmsalade,1/2 cupje margarine /roomboter,1/2 gekookt ei,1/2 haring,1/2 knäckebröd/beschuit/toast,1/2 kroket/ pizzapunt,"1/2 mini twix,mars,snicker,mailky way",1/2 omelet/roerei,1/2 pannekoek,1/2 pindakaas,1/2 plak cake,1/2 plak kaas,1/2 plak vleeswaar,1/2 portie aardappelen/aardappelpuree,1/2 portie appelmoes/vruchtenmoes,1/2 portie bami,1/2 portie cornflakes,1/2 portie fruit,1/2 portie gebakken aardappelen,1/2 portie gehakt of kip in pindasaus,1/2 portie gewelde pruimen,1/2 portie groente,1/2 portie jus,1/2 portie macaroni/spaghetti,1/2 portie makreel,1/2 portie mixmaaltijd,1/2 portie muesli,1/2 portie nagerecht,1/2 portie nasi,1/2 portie rauwkost,1/2 portie stamppot zonder vlees,1/2 portie vis,1/2 portie vlees,1/2 portie witte rijst/zilvervliesrijst,1/2 roomijs,1/2 smeerkaas,1/2 snee brood zonder korst,1/2 snee ontbijtkoek,1/2 snee wit-bruin-volkoren-krente-roggebrood,1/2 zoet beleg,1/4 waterijs,1/4 bakje eiersalade,1/4 bakje zalmsalade,1/4 cupje margarine /roomboter,1/4 gekookt ei,1/4 knäckebröd/beschuit/toast,1/4 kroket/ pizzapunt,1/4 loempia/ kip/ham,"1/4 mini twix,mars,snicker,mailky way",1/4 omelet/roerei,1/4 pakje sultana/ balisto,1/4 pannekoek,1/4 pindakaas,1/4 plak cake,1/4 plak kaas,1/4 plak vleeswaar,1/4 portie aardappelen/aardappelpuree,1/4 portie appelmoes/vruchtenmoes,1/4 portie bami,1/4 portie cornflakes,1/4 portie fruit,1/4 portie gebakken aardappelen,1/4 portie gewelde pruimen,1/4 portie groente,1/4 portie jus,1/4 portie macaroni/spaghetti,1/4 portie mixmaaltijd,1/4 portie muesli,1/4 portie nagerecht,1/4 portie nasi,1/4 portie ragout met kalfsvlees,1/4 portie rauwkost,1/4 portie stamppot zonder vlees,1/4 portie vis,1/4 portie vlees,1/4 portie witte rijst/zilvervliesrijst,1/4 portie worst,1/4 roomijs,1/4 smeerkaas,1/4 snee brood zonder korst,1/4 snee ontbijtkoek,1/4 snee wit-bruin-volkoren-krente-roggebrood,1/4 zoet beleg,2 vissticks,3/4 waterijs,3/4 bakje eiersalade,3/4 cupje margarine /roomboter,3/4 gekookt ei,3/4 knäckebröd/beschuit/toast,3/4 kroket/ pizzapunt,3/4 omelet/roerei,3/4 plak cake,3/4 plak kaas,3/4 plak vleeswaar,3/4 portie aardappelen/aardappelpuree,3/4 portie appelmoes/vruchtenmoes,3/4 portie bami,3/4 portie cornflakes,3/4 portie fruit,3/4 portie gebakken aardappelen,3/4 portie groente,3/4 portie jus,3/4 portie macaroni/spaghetti,3/4 portie mixmaaltijd,3/4 portie muesli,3/4 portie nagerecht,3/4 portie rauwkost,3/4 portie spritsbrokken,3/4 portie stamppot zonder vlees,3/4 portie vis,3/4 portie vlees,3/4 portie witte rijst/zilvervliesrijst,3/4 roomijs,3/4 smeerkaas,3/4 snee brood zonder korst,3/4 snee wit-bruin-volkoren-krente-roggebrood,3/4 zoet beleg,abciximab,able - studie,access - studie,acenocoumarol,acetazolamide,acetrizoaat meglumine,acetylcysteine,acetylsalicylzuur,aciclovir,aciclovir oculair,actrapid,adenosine,adrenaline,alarm bleeding - studie,albumine,alendroninezuur,alfacalcidol,alfuzosine,allopurinol,alprazolam,alteplase,amfotericine b,amfotericine b in

In [ ]:
# merge
df_stat_01_add = pd.concat([df_stat_01_all,df_stat_01],axis=0)
df_stat_01_add["groupHPD"] = [0,1,2,3]
# Remove drugs that are less than 10% of the total SA-AKI patients
first_row = df_stat_01_add.iloc[0]
columns_to_drop = first_row[first_row <= 10].index
df_stat_01_add = df_stat_01_add.drop(columns=columns_to_drop)
df_stat_01_add["groupHPD"] = [0,1,2,3]
df_stat_01_add = df_stat_01_add[df_stat_01_add.columns[df_stat_01_add.iloc[0].argsort()[::-1]]]
df_stat_01_add.to_csv("aumc_drug_useage.csv",index=False)
df_stat_01_add.T

,0,1,2,3
drukzak,96.701787,95.921986,96.928571,97.260274
"nacl 0,9% spuit",92.670637,94.148936,91.857143,94.063927
"nacl 0,45%/glucose 2",88.410444,89.007092,88.214286,88.127854
magnesiumsulfaat,88.273019,90.602837,87.214286,89.041096
propofol,88.089785,85.638298,88.571429,91.324201
fentanyl,84.241869,88.652482,81.214286,92.237443
nacl 0,80.897847,85.106383,78.214286,87.214612
nadroparine,79.111315,79.432624,81.428571,63.470320
noradrenaline,78.744847,86.170213,73.571429,92.694064
kaliumchloride,75.858910,80.673759,74.500000,72.146119


In [ ]:
#the top10 drugs with the differences in subtypes C2 and C3
row_difference = (df_stat_01_add.iloc[3] - df_stat_01_add.iloc[2]).abs()
top_10_columns = row_difference.nlargest(10).index
df_stat_01_add[top_10_columns]

,hydrocortison,natrium bicarbonaat 8,glucose,amiodaron onderhoudsdosis,amiodaron oplaaddosis,fytomenadion,paracetamol,enoximon,midazolam,kaliumchloride drank
0,42.098030,11.818598,40.677966,27.118644,24.874027,21.209345,73.889143,26.248282,61.245992,43.197435
1,54.787234,21.099291,55.319149,36.879433,33.510638,31.205674,72.340426,37.056738,70.035461,46.808511
2,32.857143,4.285714,31.857143,20.214286,18.785714,14.714286,77.285714,19.642857,55.357143,44.714286
3,68.493151,36.073059,59.360731,46.118721,41.552511,36.986301,56.164384,40.639269,76.255708,24.200913
